# KNeighborsClassifier for Cryotherapy dataset

Hello!

I did a little analysis of the «KNeighborsClassifier» model

## About dataset

This is Cryotherapy data<br>
I had downloaded it from UCI: https://archive.ics.uci.edu<br>
Link to this DataSet: https://archive.ics.uci.edu/ml/datasets/Cryotherapy+Dataset+

There are no description about DataSet

And i had understood it as i can :]

There are 6 features about pacients and 90 objects

1. «sex» → encoded male or female by 1 and 2 numbers. But i really dont know what number mean ^^
2. «age» → just age of pacients in INT variables
3. «Time» → time in minutes (i think) of procedure duration
4. «Number_of_Warts» → numder of warts im pacient
5. «Type» → type of warts. There are three types. But the second type is the rarest
6. «Area» → area of warts summary in mm (i think)

## First look

The data was in .xlsx format when i had download it

I transferred them to .csv with the help of LibreOffice.

Load the module in the standard way, load the data and look at the first 10 lines

In [1]:
import pandas as pd

data = pd.read_csv('Cryotherapy.csv')
data.head(10)

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
0,1,35,12.00,5,1,100,0
1,1,29,7.00,5,1,96,1
2,1,50,8.00,1,3,132,0
3,1,32,11.75,7,3,750,0
4,1,67,9.25,1,1,42,0
5,1,41,8.00,2,2,20,1
6,1,36,11.00,2,1,8,0
7,1,59,3.50,3,3,20,0
8,1,20,4.50,12,1,6,1
9,2,34,11.25,3,3,150,0


Good!

Data loaded correctly. Go ahead

Lets check shape of our data

In [2]:
data.shape

(90, 7)

OK! We have 90 rows and 7 columns

This is small, but not bad for a small illustrative example :3

Now you need to look at the information about the data using the function describe()

In [3]:
data.describe()

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,1.477778,28.600000,7.666667,5.511111,1.700000,85.833333,0.533333
std,0.502304,13.360852,3.406661,3.567155,0.905042,131.733153,0.501683
min,1.000000,15.000000,0.250000,1.000000,1.000000,4.000000,0.000000
25%,1.000000,18.000000,4.562500,2.000000,1.000000,20.000000,0.000000
50%,1.000000,25.500000,8.500000,5.000000,1.000000,70.000000,1.000000
75%,2.000000,35.000000,10.687500,8.000000,3.000000,100.000000,1.000000
max,2.000000,67.000000,12.000000,12.000000,3.000000,750.000000,1.000000


We see that in the set there are no NA values. It's good

But there are several problems:<br>
1. «sex» column is not bool<br>
2. «sex» column has unaccustomed values — 1 or 2 (not 0 or 1)<br>
3. «Type» column is not «category» type<br>
4. «Result_of_Treatment» is not «bool» type<br>

## Prepearing data

Lets fix it!

In [4]:
data['sex'].replace(2,0)
data['sex'] = data['sex'].astype(bool)
data['Type'] = data['Type'].astype('category')
data['Result_of_Treatment'] = data['Result_of_Treatment'].astype(bool)
data.dtypes

sex                        bool
age                       int64
Time                    float64
Number_of_Warts           int64
Type                   category
Area                      int64
Result_of_Treatment        bool
dtype: object

Now looks better!

In [5]:
data.describe()

,age,Time,Number_of_Warts,Area
count,90.000000,90.000000,90.000000,90.000000
mean,28.600000,7.666667,5.511111,85.833333
std,13.360852,3.406661,3.567155,131.733153
min,15.000000,0.250000,1.000000,4.000000
25%,18.000000,4.562500,2.000000,20.000000
50%,25.500000,8.500000,5.000000,70.000000
75%,35.000000,10.687500,8.000000,100.000000
max,67.000000,12.000000,12.000000,750.000000


Make two DataFrame with features and with target

In [6]:
Y = data['Result_of_Treatment']
X = data.drop('Result_of_Treatment', axis=1)

Load KNC and train_test_split to divide set on
1. X and Y for train model
2. X and Y for test model

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.22, random_state=11)

I choose test_size 22% to make 20 rows in test data

If model will do just one error — accuracy will not change by more 5%

I think, its good idea :]

Lets check shapes of X_train and X_test

In [8]:
print('Shape of X_train : {}'.format(X_train.shape))
print('Shape of X_test : {}'.format(X_test.shape))

Shape of X_train : (70, 6)
Shape of X_test : (20, 6)


Excelent!

## Time to test KNC >:]

### Check nude model \*\_\_\_\_\_\_\*

Lets try without parametres :)

In [22]:
knn_simple = KNeighborsClassifier()
knn_simple.fit(X_train, y_train)

print("Score with Train Data: {}".format(knn_simple.score(X_train, y_train)))
print("Score with Test Data: {}".format(knn_simple.score(X_test, y_test)))

Score with Train Data: 0.8571428571428571
Score with Test Data: 0.75


OK

«Train» result is better then «Test». Its normally :)

### First experement with «n»

I did a loop with «i» from 1 to 10

«i» is value of «n_neighbors»

I wanted to find best parameter of «n»

Go watch result!

In [31]:
# =======================
# Make an empty variables
# =======================

best_n_train = 0
best_n_test = 0

best_score_train = 0
best_score_train_test = 0

best_score_test = 0
best_score_test_train = 0

# ==========
# Start loop
# ==========

for i in range (1,11):
    knn_i = KNeighborsClassifier(n_neighbors=i)
    knn_i.fit(X_train, y_train)
    
    score_train = knn_i.score(X_train, y_train)
    score_test = knn_i.score(X_test, y_test)
    
    if score_train > best_score_train:
        best_score_train = score_train
        best_score_train_test = score_test
        best_n_train = i
    
    if score_test > best_score_test:
        best_score_test_train = score_train
        best_score_test = score_test
        best_n_test = i
        
# ================
# Show the results
# ================

print("= Set of bests n:")
print("Best n of Train: {}".format(best_n_train))
print("Best n of Test: {}".format(best_n_test))
print("\n")

print("= Set of results of model with bests train scores:")
print("Best Train score with n from Best Train Model: {}".format(best_score_train))
print("Best Test score with n from Best Train Model: {}".format(best_score_train_test))
print("\n")

print("= Set of results of model with bests test scores:")
print("Best Train score with n from Best Test Model: {}".format(best_score_test_train))
print("Best Test score with n from Best Test Model: {}".format(best_score_test))
print("\n")

= Set of bests n:
Best n of Train: 1
Best n of Test: 4


= Set of results of model with bests train scores:
Best Train score with n from Best Train Model: 1.0
Best Test score with n from Best Train Model: 0.8


= Set of results of model with bests test scores:
Best Train score with n from Best Test Model: 0.9
Best Test score with n from Best Test Model: 0.85




The results show some things:
1. KNC-Model with n=1 make 100% result with train data (its ok)
2. KNC Model with best train result — is not matter good score with train data (80%)
3. KNC-Model with best test result make train result some lower, but test resul some better

### Simple check «weights» with n = best test

Lest check KNC-Model with best test n and change another parameter — «weights = 'distance'»

In [11]:
knn_best_dist = KNeighborsClassifier(n_neighbors=best_n_test, weights='distance')
knn_best_dist.fit(X_train, y_train)

print(knn_best_dist.score(X_train, y_train))
print(knn_best_dist.score(X_test, y_test))

1.0
0.8


OK

Model made train results better, but test results lower

### Check parameter «algorithm» with n = best test

Well

Go next and start play with parameter «algoritm» :)

In [36]:
knn_best_balltree = KNeighborsClassifier(n_neighbors=best_n_test, algorithm='ball_tree')
knn_best_balltree.fit(X_train, y_train)

print("Train score: {}".format(knn_best_balltree.score(X_train, y_train)))
print("Test score: {}".format(knn_best_balltree.score(X_test, y_test)))

Train score: 0.9
Test score: 0.85


In [37]:
knn_best_kdtree = KNeighborsClassifier(n_neighbors=best_n_test, algorithm='kd_tree')
knn_best_kdtree.fit(X_train, y_train)

print("Train score: {}".format(knn_best_kdtree.score(X_train, y_train)))
print("Test score: {}".format(knn_best_kdtree.score(X_test, y_test)))

Train score: 0.9
Test score: 0.85


In [38]:
knn_best_brute = KNeighborsClassifier(n_neighbors=best_n_test, algorithm='brute')
knn_best_brute.fit(X_train, y_train)

print("Train score: {}".format(knn_best_brute.score(X_train, y_train)))
print("Test score: {}".format(knn_best_brute.score(X_test, y_test)))

Train score: 0.9
Test score: 0.85


Like we see: this parametres do not affect in this case ^^

### Check «weights» with nude model

In [39]:
knn_simple_dist = KNeighborsClassifier(weights='distance')
knn_simple_dist.fit(X_train, y_train)

print("Train score: {}".format(knn_simple_dist.score(X_train, y_train)))
print("Test score: {}".format(knn_simple_dist.score(X_test, y_test)))

Train score: 1.0
Test score: 0.85


Good!

It shows best results now :)

But default value of n — is 5. I think, we should play with this model more to find something else

### Check «algoritm» with nude model

In [40]:
knn_simple_kdtree = KNeighborsClassifier(algorithm='ball_tree')
knn_simple_kdtree.fit(X_train, y_train)

print("Train score: {}".format(knn_simple_kdtree.score(X_train, y_train)))
print("Test score: {}".format(knn_simple_kdtree.score(X_test, y_test)))

Train score: 0.8571428571428571
Test score: 0.75


In [41]:
knn_simple_kdtree = KNeighborsClassifier(algorithm='kd_tree')
knn_simple_kdtree.fit(X_train, y_train)

print("Train score: {}".format(knn_simple_kdtree.score(X_train, y_train)))
print("Test score: {}".format(knn_simple_kdtree.score(X_test, y_test)))

Train score: 0.8571428571428571
Test score: 0.75


In [42]:
knn_simple_brute = KNeighborsClassifier(algorithm='brute')
knn_simple_brute.fit(X_train, y_train)

print("Train score: {}".format(knn_simple_brute.score(X_train, y_train)))
print("Test score: {}".format(knn_simple_brute.score(X_test, y_test)))

Train score: 0.8571428571428571
Test score: 0.75


In this example — parameter changes do not change the result

### Loop to find best model with «weights» parameter is 'algorithm'

In [44]:
# ===============
# Empty variables
# ===============

best_n_train = 0
best_n_test = 0

best_score_train = 0
best_score_train_test = 0

best_score_test = 0
best_score_test_train = 0

# ====
# Loop
# ====

for i in range (1,11):
    knn_i = KNeighborsClassifier(n_neighbors=i,
                                 weights='distance')
    knn_i.fit(X_train, y_train)

    score_train = knn_i.score(X_train, y_train)
    score_test = knn_i.score(X_test, y_test)

    if score_train > best_score_train:
        best_score_train = score_train
        best_score_train_test = score_test
        best_n_train = i

    if score_test > best_score_test:
        best_score_test_train = score_train
        best_score_test = score_test
        best_n_test = i

# ============     
# Show results
# ============

print("= Set of bests n:")
print("Best n of Train: {}".format(best_n_train))
print("Best n of Test: {}".format(best_n_test))
print("\n")

print("= Set of results of model with bests train scores:")
print("Best Train score with n from Best Train Model: {}".format(best_score_train))
print("Best Test score with n from Best Train Model: {}".format(best_score_train_test))
print("\n")

print("= Set of results of model with bests test scores:")
print("Best Train score with n from Best Test Model: {}".format(best_score_test_train))
print("Best Test score with n from Best Test Model: {}".format(best_score_test))
print("\n")

= Set of bests n:
Best n of Train: 1
Best n of Test: 3


= Set of results of model with bests train scores:
Best Train score with n from Best Train Model: 1.0
Best Test score with n from Best Train Model: 0.8


= Set of results of model with bests test scores:
Best Train score with n from Best Test Model: 1.0
Best Test score with n from Best Test Model: 0.85




## Findings

### Main

Best model for this data set is
- n_neighbors = 3
- weights = 'distance'

I think, this is not full analisys that i can make
Its will better, if i add a graphics and more visual information

## PS

- This DataSet is some small to do great analisys
- The results are so much depended on random seed of «train_test_split» and i had checked a lot of numbers, before found this one

# Thanks for watching :3
## Good luck!

![title](bottom_img.gif)